## Config

In [1]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

## Log In

In [2]:
# Authenticate to Spotify
spotify, userID, user_token = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Collect new artists

In [3]:
# Collect names of new artists
artistsNew = fetchMusicFestivalWizard()

## Process new artists

In [5]:
# Filter out known artists who we have already listened to
if os.path.exists("artistsKnown.csv"):
    artistsKnown = pd.read_csv('artistsKnown.csv', index_col=0)
else:
    artistsKnown = pd.DataFrame(columns=['artistName', 'artistID', 'artistPopularity', 'artistFollowers', 'artistGenre'])
artistsNew = artistsNew.loc[~artistsNew.artistName.isin(artistsKnown['artistName'])]
# Search Spotify for new artist info
artistsNew = artistsNew.apply(fetchArtistInfo, axis=1)
# Ignore artists who did not turn up in Spotify
artistsNew = artistsNew.dropna(subset='artistID')
# Drop artists whose name was not an exact match with a known artist but who have the same artistID
artistsNew = artistsNew.loc[~artistsNew.artistID.isin(artistsKnown['artistID'])]

,artistName


## Get recent songs

In [83]:
# Get their top three songs in last 5 years
songURIs = fetchTopSongs(artistsNew['artistID'], maxAge=5)
# How many songs are in this round?
print(f"There are {len(songURIs)} songs about to be added")

There are 6 about to be added


## Put songs in playlist

In [114]:
# Warning: This will make a new playlist
playlistID = getPlaylistID('New Artists to Explore')
# Add songs to the playlist
addToPlaylist(songURIs, playlistID)

## Update the list of known artists

In [126]:
# Append artistsNew to artistsKnown.csv 
allArtists = pd.concat([artistsKnown, artistsNew], axis=0, ignore_index=True)
allArtists.to_csv('artistsKnown.csv')

## Create playlist to drop good songs into

In [118]:
# Cull exploreArtists into ArtistHasPotential playlist
winnersPlaylist = getPlaylistID('New Artists I Liked')
print(f'Now drop songs you like into this playlist: {winnersPlaylist}')

Now drop songs you like into this playlist: 5bZPe8Gkc9jUADiDXXlm4V
